In [1]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *

creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',)]
42


In [2]:
alertP = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)
alertP1=alertP.copy()
alertP1
alertP1.info()
alertP1.head()

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10150 entries, 0 to 10149
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_DOENTE            10150 non-null  object
 1   PROCESSO             10150 non-null  object
 2   COD_REFERENCIA       10150 non-null  object
 3   COD_PZ               10150 non-null  object
 4   COD_UNID_SAUDE_PROV  10150 non-null  int64 
 5   UNID_PROV            10150 non-null  object
 6   TIPO_UNID            10150 non-null  object
 7   PROVENIENCIA         10150 non-null  object
 8   COD_CTH_PRIOR        10150 non-null  object
 9   CTH_PRIOR            10150 non-null  object
 10  COD_MOTIVO_RECUSA    10150 non-null  int64 
 11  DES_MOTIVO_RECUSA    10150 non-null  object
 12  COD_ESPECIALIDADE    10150 non-null  int64 
 13  DES_ESPECIALIDADE    10150 non-null  object
 14  agrupadora           10150 non-null  object
 15  OUTRA_ENTIDADE       10150 non-null  int64 
 16  DATA

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,DATA_MARCACAO,DATA_REALIZACAO,OBSERVACOES,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,21/05/2015,12/06/2015,,11,2014,21/05/2015,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an..."
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,,...,27/04/2012,02/07/2012,,4,2012,23/04/2012,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,,...,26/09/2014,13/01/2015,,9,2014,26/09/2014,13/01/2015,1,2015,cefaleias
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,,...,28/10/2013,03/12/2013,,10,2013,25/10/2013,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ..."
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,,,,...,,,Conforme e-mail de 30/11/2015,2,2013,26/02/2013,26/02/2013,2,2013,Sem relatorio clinico


In [3]:
#changing columns with date to a datetime
date_format_alertP1(alertP1)
#change the blank values in proveniencia and priority to unknown
replace_blank(alertP1)
#create a column with the output of a referral and select the refused and accepted ones
alertP1=result(alertP1)#changing columns with date to a datetime
date_format_alertP1(alertP1)
#change the blank values in proveniencia and priority to unknown
replace_blank(alertP1)
#create a column with the output of a referral and select the refused and accepted ones
alertP1=result(alertP1)
class_area(alertP1)
text_length(alertP1)
referral_steps(alertP1)
alertP1=unit(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
alertP1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9260 entries, 0 to 10149
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_DOENTE            9260 non-null   object        
 1   PROCESSO             9260 non-null   object        
 2   COD_REFERENCIA       9260 non-null   object        
 3   COD_PZ               9260 non-null   object        
 4   COD_UNID_SAUDE_PROV  9260 non-null   int64         
 5   UNID_PROV            9260 non-null   object        
 6   TIPO_UNID            9260 non-null   object        
 7   PROVENIENCIA         9260 non-null   object        
 8   COD_CTH_PRIOR        9260 non-null   object        
 9   CTH_PRIOR            9260 non-null   object        
 10  COD_MOTIVO_RECUSA    9260 non-null   int64         
 11  DES_MOTIVO_RECUSA    9260 non-null   object        
 12  COD_ESPECIALIDADE    9260 non-null   int64         
 13  DES_ESPECIALIDADE    9260 non-nu

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
alertP1.head()

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,trata data recusa,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,unit
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,21/05/2015,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an...",1,inside area,701,1,USF
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,unknown,...,23/04/2012,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...,1,inside area,363,1,USF
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,unknown,...,26/09/2014,13/01/2015,1,2015,cefaleias,1,outside area,9,1,HOSP
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,25/10/2013,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ...",1,inside area,417,1,USF
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,unknown,,unknown,...,26/02/2013,26/02/2013,2,2013,Sem relatorio clinico,0,outside area,22,1,HOSP


In [24]:
all=alertP1[["ID_DOENTE","DATA_RECEPCAO","COD_MOTIVO_RECUSA","DES_MOTIVO_RECUSA","DES_ESPECIALIDADE","result","area","text_length","step","unit"]]
all["total"]=1
all["result"]=all["result"].astype(int)
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9260 entries, 0 to 10149
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID_DOENTE          9260 non-null   object        
 1   DATA_RECEPCAO      9260 non-null   datetime64[ns]
 2   COD_MOTIVO_RECUSA  9260 non-null   int64         
 3   DES_MOTIVO_RECUSA  9260 non-null   object        
 4   DES_ESPECIALIDADE  9260 non-null   object        
 5   result             9260 non-null   int64         
 6   area               9260 non-null   object        
 7   text_length        9260 non-null   int64         
 8   step               9260 non-null   object        
 9   unit               9260 non-null   object        
 10  total              9260 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 868.1+ KB


/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_93332/2006963749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["total"]=1
/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_93332/2006963749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["result"]=all["result"].astype(int)


In [16]:
unit=all.groupby("unit").sum()
unit["acc_rate"]=unit["result"]/unit["total"]

print(unit)

      COD_MOTIVO_RECUSA  result  text_length  total  acc_rate
unit                                                         
CS                13780    1330       175982   1764  0.753968
HOSP              18803    1887       101723   3735  0.505221
UCSP               4995     499        64497    696  0.716954
USF               23181    2538       398282   3065  0.828059


In [8]:
unit_t=all[all["text_length"]>0]
unit_t=unit_t.groupby("unit").sum()
unit_t["acc_rate"]=unit_t["result"]/unit_t["total"]
print(unit_t)


      result  text_length  total  acc_rate
unit                                      
CS       233       175982    301  0.774086
HOSP     340       101723    651  0.522273
UCSP      87        64497    117  0.743590
USF      439       398282    530  0.828302


In [9]:
#text lenght >1, 
spec=all[all["text_length"]>0]
spec=spec.groupby("DES_ESPECIALIDADE").sum()
spec["acc_rate"]=spec["result"]/spec["total"]
print(spec)





                                     result  text_length  total  acc_rate
DES_ESPECIALIDADE                                                        
APOIO DOENCAS NEUROLOGICAS CRONICAS       1          201      1  1.000000
DOENCAS DO MOVIMENTO                     50        18635     50  1.000000
DVC                                       9         3930      9  1.000000
EPILEPSIA                                69        13440     69  1.000000
NEUROLOGIA                              904       667935   1402  0.644793
NEUROLOGIA - CEFALEIAS                    1           95      1  1.000000
NEUROLOGIA - ESCLEROSE MULTIPLA           7         2178      7  1.000000
NEUROLOGIA DO COMPORTAMENTO              58        34070     60  0.966667


In [10]:
epilepsia=all[all["DES_ESPECIALIDADE"]=="EPILEPSIA"]
epilepsia=epilepsia[epilepsia["text_length"]>0]
epilepsia["result"].value_counts()
epilepsia

1    69
Name: result, dtype: int64

,ID_DOENTE,DATA_RECEPCAO,DES_ESPECIALIDADE,result,area,text_length,step,unit,total
2,ADCFD,2014-09-26,EPILEPSIA,1,outside area,9,1,HOSP,1
102,SHFZAC,2014-04-02,EPILEPSIA,1,outside area,15,1,HOSP,1
109,SZCGAF,2016-12-16,EPILEPSIA,1,outside area,10,2,HOSP,1
125,ZBHDSZ,2014-09-19,EPILEPSIA,1,outside area,196,2,HOSP,1
133,GAFGD,2014-10-27,EPILEPSIA,1,outside area,53,3+,HOSP,1
...,...,...,...,...,...,...,...,...,...
1548,FGFFDE,2013-11-29,EPILEPSIA,1,inside area,729,1,CS,1
1571,FGCCSA,2013-01-02,EPILEPSIA,1,outside area,9,1,HOSP,1
1629,AHDEFH,2014-04-29,EPILEPSIA,1,outside area,9,1,HOSP,1
1692,AHGADG,2012-04-18,EPILEPSIA,1,outside area,825,1,USF,1


In [32]:

b=all[all["text_length"]>0]
c=b.groupby(["DES_ESPECIALIDADE","COD_MOTIVO_RECUSA","DES_MOTIVO_RECUSA"])["result","total"].sum()
c.reset_index()
6254/9260

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_93332/3405078654.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  c=b.groupby(["DES_ESPECIALIDADE","COD_MOTIVO_RECUSA","DES_MOTIVO_RECUSA"])["result","total"].sum()


,DES_ESPECIALIDADE,COD_MOTIVO_RECUSA,DES_MOTIVO_RECUSA,result,total
0,APOIO DOENCAS NEUROLOGICAS CRONICAS,0,,1,1
1,DOENCAS DO MOVIMENTO,0,,38,38
2,DOENCAS DO MOVIMENTO,15,Consulta Marcada. O Doente Desistiu,1,1
3,DOENCAS DO MOVIMENTO,53,Falta Injustificada do Utente,11,11
4,DVC,0,,8,8
5,DVC,15,Consulta Marcada. O Doente Desistiu,1,1
6,EPILEPSIA,0,,57,57
7,EPILEPSIA,12,Marcada em Subsequentes,1,1
8,EPILEPSIA,13,Marcada sem Referencia,1,1
9,EPILEPSIA,53,Falta Injustificada do Utente,10,10


0.675377969762419

In [ ]:
#double check if th
#with text how many have data recepcao and that kind of datas. Compare it with full sample and ones in. 
the code